<a href="https://colab.research.google.com/github/matthewmachnowski/artificial-intelligence-uw/blob/main/Taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taxi

Środowisko Taxi przypomina pracę taksówkarza w mieście. Złożone jest z kwadratowej planszy podzielonej na $n \times n$ kwadratów jednostkowych, każdy z nich może być przejezdny bądź nie. Mamy wyróżnione cztery rogi, nazwane odpowiednio R, G, Y i B. Nasz agent pojawia się w losowym miejscu, a jego celem jest obsługa zlecenia przewiezienia klienta między zadanymi rogami. Powinien zrobić to jak najsprawniej. Przykładowa plansza wygląda następująco:

![](env.png)
<center>Agent znajduje się w żółtym polu i za zadanie ma przewieźć klienta z R (niebieskie) do B (fioletowe).</center>

W każdym momencie, nasz agent będzie mógł wykonać jedną z sześciu akcji:
<ol start="0">
  <li>Jedź w dół,</li>
  <li>Jedź w górę,</li>
  <li>Jedź w prawo,</li>
  <li>Jedź w lewo,</li>
  <li>Weź klienta,</li>
  <li>Wysadź klienta.</li>
</ol>

Każda z akcji 0-3 daje nam zysk -1 (kosztuje 1), akcje 4-5 dają zysk 0, gdy są w wykonane w poprawnym momencie, i -10, gdy nie. Wykonanie zadania daje dużą nagrodę: 10n.


## Zadanie:
- zaimplementować Q-learning
- dobrać parametry
- sprawdzić, dla jakich rozmiarów planszy nasza metoda znajduje rozwiązanie (pośród 6, 10, 20 i 100)

## Opis stanu gry:
- stan gry jest reprezentowany przez jedną liczbę

In [1]:
try:
    import gym
except:
    !apt-get install cmake swig
    !pip install gym[all]

data_url = 'https://www.mimuw.edu.pl/~mm319369/priv/d73890416bec03ff3e2b3756af8c941c/task1-data.zip'

def download_zip(url):
    from zipfile import ZipFile
    try:
        from urllib import urlretrieve
    except:
        from urllib.request import urlretrieve
    from tempfile import mktemp

    filename = mktemp('.zip')
    name, hdrs = urlretrieve(data_url, filename)
    thefile = ZipFile(filename)
    thefile.extractall('')
    thefile.close()
    
import os

if not os.path.exists('taxi_custom.py'):
    download_zip(data_url)
    print("Done downloading")
else:
    print("Files already exist")

Done downloading


In [2]:
import numpy as np
import gym
import random
from collections import defaultdict
from taxi_custom import TaxiBuilder

TB = TaxiBuilder()
SOLVE_SCORE_FOR_6 = 43

In [3]:
class QPlayer(object):
    def __init__(self, num_of_actions, num_of_states=None):
        # Uzupełnij wartości parametrów
        self.lr = 0.05 #learning rate: 0.0 - 1.0
        self.gamma = 0.99 #gamma (discount factor): 0.0 - 1.0
        self.epsilon_start = 1 #wartość początkowa dla epsilon-greedy: 0.0 - 1.0
        self.epsilon_min = 0.01 #wartosc koncowa dla epsilon-greedy: 0.0 - 1.0
        self.epsilon_decay = 0.99999 #szybkosc wygaszania epsilon: 0.9 - 0.999999...

        self.num_of_actions = num_of_actions
        self.num_of_states = num_of_states
        
        self.Q = defaultdict(lambda: np.zeros(self.num_of_actions))
    
    def encode_state(self, state):
        # ta metoda moze byc wykorzystana, jezeli trzeba przetworzyć stan gry,
        # np. jeżeli trzeba dokonać kwantyzacji 
        return state
    
    def train(self, env, steps):
        # Metoda 'train' przyjmuje jako parametry zainicjowane srodowisko ai gym (env) oraz 
        # liczbę kroków uczenia (steps). Wewnątrz tej metody należy zaimplementować uczenie Q-learning
        # Należy wykorzystac dobrane parametry uczenia (lr, gamma,...) zdefiniowane w __init__
        
        epsilon = self.epsilon_start
        rewards = []
        counter = 0

        while steps > 0: 
          state = env.reset()
          end = False
          counter += 1
          score = 0

          while not end and steps > 0:

            n = np.random.random(1)[0]

            if n < epsilon:
              action = env.action_space.sample() # losowa akcja 
            else:
              action = np.argmax(self.Q[state]) # najlepsza akcja 

            new_state, reward, end, _ = env.step(action)

            if end:
              self.Q[state][action] = self.lr * reward + (1 - self.lr) * self.Q[state][action]
            else:
              self.Q[state][action] += self.lr * (reward + self.gamma * np.max(self.Q[new_state]) - self.Q[state][action])

            if steps % 100_000 == 0:
              if not rewards:
                self.data = np.array([counter, steps, 0, epsilon])
                print(counter, steps, "\t", epsilon, sep='\t')
              else:
                print(counter, steps, np.mean(rewards), epsilon, sep='\t')
                self.data = np.append(self.data, [counter, steps, np.mean(rewards), epsilon])
                rewards = []

            state = new_state
            steps -= 1;
            score += reward
            if epsilon > self.epsilon_min:
                epsilon *= self.epsilon_decay
            
        rewards.append(score)
    
    def play(self, state):
        # Ta metoda zwraca (wyuczoną) akcje dla danego stanu
        # Uzywana do 'grania' już wytrenowanym agentem
        s = self.encode_state(state)
        return np.argmax(self.Q[s])
        
        

## Rozwiązanie na planszy 6 – obowiązkowe

In [4]:
# Bierzemy planszę o boku 6
_, env = TB.get_instance(6)
print("OBSERVATION SPACE:", env.observation_space)
print("ACTION SPACE:", env.action_space)

OBSERVATION SPACE: Discrete(8192)
ACTION SPACE: Discrete(6)


In [5]:
player = QPlayer(num_of_actions=env.action_space.n, num_of_states=env.observation_space.n)

#Trenujemy
player.train(env, 1_000_000)

1	1000000			1
1694	900000			0.3678776017682482
3735	800000			0.13533392988275578
7519	700000			0.04978632156314083
13164	600000			0.01831527257751119
19033	500000			0.009999971601096055
24928	400000			0.009999971601096055
30824	300000			0.009999971601096055
36732	200000			0.009999971601096055
42612	100000			0.009999971601096055


In [6]:
def evaluate_player(player, env):
     
    scores = []
    for i in range(1000):
        score = 0
        s = env.reset()
        while(True):
            a = player.play(s)
            s, r, end, _ = env.step(a)
            score += r
            if end:
                break
                
        #print(i, score)
        scores.append(score)
    
    return np.mean(scores)

#Testowanie
_, new_env = TB.get_instance(6)
test_player = player

score = evaluate_player(test_player, new_env)
print("WYNIK:", score)

assert score >= SOLVE_SCORE_FOR_6, "Trzeba jeszcze popracować..."
print("GRA ROZWIĄZANA!")

WYNIK: 43.948
GRA ROZWIĄZANA!


## Uczenie na większych planszach
Sprawdź, czy jesteśmy w stanie nauczyć agenta grać na planszach o większych rozmiarach. Jeśli nie, co stoi na przeszkodzie?

In [7]:
_, env = TB.get_instance(10) # 6, 10, 20, 100
print("OBSERVATION SPACE:", env.observation_space)
print("ACTION SPACE:", env.action_space)

OBSERVATION SPACE: Discrete(8192)
ACTION SPACE: Discrete(6)


## Zdaj krótki raport w polu poniżej.

Działa do plansz o boku do 10 w sensownym czasie.
Potem przestrzeń stanów jest zbyt duża, by dojść do rozwiązania w sposób losowy.